In [1]:
!sudo apt update
!sudo apt -y install build-essential
!pip install networkx node2vec python-Levenshtein sklearn

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.1 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [488 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [790 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1,037 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [101 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1,275 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:12 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu fo

In [2]:
import pandas as pd

index = pd.read_csv('https://ameo.dev/spotify_items.csv.gz', compression = 'gzip')

In [3]:
index.head()

,id,spotify_id
0,146079057,0000BlyH0BpSivfg8y7QSb
1,146505418,0001cekkfdEBoMlwVQvpLg
2,88525725,0001wHqxbF2YYRQxGdbyER
3,116519192,0001ZVMPt41Vwzt1zsmuzp
4,115213790,0002XY9y3JhjzTZqNCqEcv


In [4]:
spotify_ids_by_id = {}

for (id, artist_spotify_id) in index.itertuples(index=False):
  spotify_ids_by_id[int(id)] = artist_spotify_id

In [5]:
ids_by_spotify_id = {}

for (id, spotify_id) in spotify_ids_by_id.items():
  ids_by_spotify_id[spotify_id] = id

In [6]:
related_artists_raw = pd.read_csv('https://ameo.dev/related_artists.csv.gz', compression = 'gzip')

In [7]:
related_artists_raw.head()

,artist_spotify_id,related_artists_json
0,1,"[""5uCktjKM1Rf9VnUvFD6j17"",""5FAjeD9eW52qvpik9Os..."
1,2,"[""3j24yoSFDD11h1OszJFfVY"",""2OgP0RGmDrXN9kOWRNf..."
2,3,"[""0UM3yh4PeDSWa9cnHJ0jgW"",""0q0K0FV5t8OnVpbIQTX..."
3,4,"[""7gjAu1qr5C2grXeQFFOGeh"",""1qcU8NabUQ5efQf8UGW..."
4,5,"[""3QpNnKSjUIIMwFei8h3eOj"",""04Y5zXsFqYRzDr6tMXj..."


In [8]:
related_artists_raw['id'] = related_artists_raw['artist_spotify_id']
related_artists_raw['artist_spotify_id'] = related_artists_raw['artist_spotify_id'].apply(lambda id: spotify_ids_by_id[int(id)])

In [9]:
related_artists_raw.head()

,artist_spotify_id,related_artists_json,id
0,4IuaUq1boR1KVcutaID4Zx,"[""5uCktjKM1Rf9VnUvFD6j17"",""5FAjeD9eW52qvpik9Os...",1
1,2UwJRAgSOi1zcLkvUNc8XL,"[""3j24yoSFDD11h1OszJFfVY"",""2OgP0RGmDrXN9kOWRNf...",2
2,4BrtGOYp8sytG9MV91DROT,"[""0UM3yh4PeDSWa9cnHJ0jgW"",""0q0K0FV5t8OnVpbIQTX...",3
3,1xU878Z1QtBldR7ru9owdU,"[""7gjAu1qr5C2grXeQFFOGeh"",""1qcU8NabUQ5efQf8UGW...",4
4,2Ub06wAIR1hERODcCkKhzx,"[""3QpNnKSjUIIMwFei8h3eOj"",""04Y5zXsFqYRzDr6tMXj...",5


In [10]:
import json

related_artists_raw['related_artists'] = related_artists_raw['related_artists_json'].apply(lambda x: json.loads(x))

In [11]:
del related_artists_raw['related_artists_json']
related_artists_raw.head()

,artist_spotify_id,id,related_artists
0,4IuaUq1boR1KVcutaID4Zx,1,"[5uCktjKM1Rf9VnUvFD6j17, 5FAjeD9eW52qvpik9OsnV..."
1,2UwJRAgSOi1zcLkvUNc8XL,2,"[3j24yoSFDD11h1OszJFfVY, 2OgP0RGmDrXN9kOWRNfEu..."
2,4BrtGOYp8sytG9MV91DROT,3,"[0UM3yh4PeDSWa9cnHJ0jgW, 0q0K0FV5t8OnVpbIQTXOh..."
3,1xU878Z1QtBldR7ru9owdU,4,"[7gjAu1qr5C2grXeQFFOGeh, 1qcU8NabUQ5efQf8UGW6C..."
4,2Ub06wAIR1hERODcCkKhzx,5,"[3QpNnKSjUIIMwFei8h3eOj, 04Y5zXsFqYRzDr6tMXj6a..."


In [20]:
import networkx as nx
graph = nx.Graph()

# load json from file
ids_to_retain = set()
with open('work/ids_to_keep.json', 'r') as f:
    parsed = json.load(f)
    for id in parsed:
        ids_to_retain.add(id)

all_related_artists = related_artists_raw[:100_000]
# Only retain ids that exist in `ids_to_retain`
all_related_artists = all_related_artists[all_related_artists['id'].isin(ids_to_retain)]

graph.add_nodes_from(all_related_artists['id'])
print(f"Total retained id count: {len(all_related_artists)}")

Total retained id count: 77555


In [21]:
for (artist_spotify_id, id, related_artists) in all_related_artists.itertuples(index=False):
  for related_artist_spotify_id in related_artists:
    related_artist_id = ids_by_spotify_id.get(related_artist_spotify_id)
    if related_artist_id is None:
      continue

    graph.add_edge(id, related_artist_id)
    graph.add_edge(related_artist_id, id)

In [22]:
from node2vec import Node2Vec

def build_model(**kwargs):
    return Node2Vec(graph, **kwargs)

def train_model(node2vec_inst, **kwargs):
    return node2vec_inst.fit(**kwargs)

In [23]:
# build_model(dimensions=4, walk_length=10, num_walks=15, workers=12)
# train_model(window=7, min_count=1, workers=14, hs=1, sg=1)

for q_index in range(0, 6):
    q = q_index * 0.5 + 0.1

    for p_index in range(0, 6):
        p = p_index * 0.5 + 0.1

        print(f'Building + training for p={p} q={q}...')

        node2vec_inst = build_model(dimensions=4, walk_length=10, num_walks=20, workers=5, p=p, q=q)
        model = train_model(node2vec_inst, window=7, min_count=1, workers=12, hs=1, sg=1)

        print(f'Training finished for p={p} q={q}, saving to output files...')
        output_file_name_base = f'work/hyperparam_search/100k_pop_filtered_corpus_p_{int(p * 10)}_q_{int(q * 10)}'

        model.wv.save_word2vec_format(f"{output_file_name_base}.w2v", f"{output_file_name_base}.vocab")

Building + training for p=0.1 q=0.1...


Computing transition probabilities:   0%|          | 0/224387 [00:00<?, ?it/s]

Generating walks (CPU: 5): 100%|██████████| 4/4 [03:32<00:00, 53.24s/it]


Training finished for p=0.1 q=0.1, saving to output files...
Building + training for p=0.6 q=0.1...


Computing transition probabilities:   0%|          | 0/224387 [00:00<?, ?it/s]

...

IOStream.flush timed out
Generating walks (CPU: 5): 100%|██████████| 4/4 [03:26<00:00, 51.57s/it]


Training finished for p=2.1 q=2.6, saving to output files...
Building + training for p=2.6 q=2.6...


Computing transition probabilities:   0%|          | 0/224387 [00:00<?, ?it/s]

Generating walks (CPU: 5): 100%|██████████| 4/4 [03:27<00:00, 51.77s/it]


Training finished for p=2.6 q=2.6, saving to output files...


In [15]:
p = 0.4
q = 1.4
node2vec_inst = build_model(dimensions=4, walk_length=10, num_walks=20, workers=5, p=p, q=q)
model = train_model(node2vec_inst, window=7, min_count=1, workers=12, hs=1, sg=1)

Computing transition probabilities:   0%|          | 0/215351 [00:00<?, ?it/s]

Generating walks (CPU: 5): 100%|██████████| 4/4 [03:25<00:00, 51.28s/it]


In [16]:
output_file_name_base = f'work/hyperparam_search/68_5k_corpus_p_{int(p * 10)}_q_{int(q * 10)}'
print(output_file_name_base)
model.wv.save_word2vec_format(f"{output_file_name_base}.w2v", f"{output_file_name_base}.vocab")

work/hyperparam_search/68_5k_corpus_p_4_q_14


In [28]:
tv_room_id = str(ids_by_spotify_id['0EFfq11AFfQOjJ0PvAvvSW'])
sleigh_bells_id = str(ids_by_spotify_id['59pWgeY26Q6yJy37QvJflh'])
red_velvet_id = str(ids_by_spotify_id['1z4g3DjTBBZKhvAroFlhOM'])
siinamota_id = str(ids_by_spotify_id['4qfQTMwNNuBKJ4rF9Lo4bD'])
current_value_id = str(ids_by_spotify_id['6B9SjvZNSQZkeJDH17oBSO'])
shiiit_id = str(ids_by_spotify_id['0RfocEzLe78RexLTeU1K2p'])
skrillex_id = str(ids_by_spotify_id['5he5w2lnU9x7JFhnwcekXX'])
kobaryo_id = str(ids_by_spotify_id['1Y81Ch90opScfpMfN17lZb'])
loona_id = str(ids_by_spotify_id['52zMTJCKluDlFwMQWmccY7'])
skylar_spence_id = str(ids_by_spotify_id['0x0u0jCVf5Jf4DNh45XPXL'])
taylor_swift_id = str(ids_by_spotify_id['06HL4z0CvFAxyc27GXpf02'])
freddie_dredd_id = str(ids_by_spotify_id['0dlDsD7y6ccmDm8tuWCU6F'])
g_herbo_id = str(ids_by_spotify_id['5QdEbQJ3ylBnc3gsIASAT5'])
fox_stevenson_id = str(ids_by_spotify_id['2BQWHuvxG4kMYnfghdaCIy'])
lorde_id = str(ids_by_spotify_id['163tK9Wjr9P9DmM0AVK7lm'])
neon_indian_id = str(ids_by_spotify_id['0KydPQPUUoTNhmiHKOg5Er'])
quok_id = str(ids_by_spotify_id['1hjJ4bsbNMmbIih3d9Pq1H'])
strfkr_id = str(ids_by_spotify_id['2Tz1DTzVJ5Gyh8ZwVr6ekU'])
beatles_id = str(ids_by_spotify_id['3WrFJ7ztbogyGnTHbHJFl2'])
drake_id = str(ids_by_spotify_id['3TVXtAsR1Inumwj472S9r4'])
anamanaguchi_id = str(ids_by_spotify_id['2UwJRAgSOi1zcLkvUNc8XL'])
ed_sheeran_id = str(ids_by_spotify_id['6eUKZXaKkcviH0Ku9w2n3V'])
frank_sinatra_id = str(ids_by_spotify_id['1Mxqyy3pSjf8kZZL4QVxS0'])
tame_impala_id = str(ids_by_spotify_id['5INjqkS1o8h1imAzPqGZBb'])
queen_id = str(ids_by_spotify_id['1dfeR4HaWDbWqFHLkxsg1d'])
ngutp_id = str(ids_by_spotify_id['0j5kVHxvTgUN4nBIPKCLRJ'])
infected_mushroom_id = str(ids_by_spotify_id['6S2tas4z6DyIklBajDqJxI'])
dither_id = str(ids_by_spotify_id['2CTPZndAY5BwdgHk8AX0B7'])
john_williams_id = str(ids_by_spotify_id['3dRfiJ2650SZu6GbydcHNb'])
dababy_id = str(ids_by_spotify_id['4r63FhuTkUYltbVAg5TQnk'])
blackbear_id = str(ids_by_spotify_id['2cFrymmkijnjDg9SS92EPM'])
chon_id = str(ids_by_spotify_id['2JFljHPanIjYy2QqfNYvC0'])
deco_id = str(ids_by_spotify_id['7kZTWx6cRLc0TSRPq1XBMP'])
dareharu_id = str(ids_by_spotify_id['4mcnvjLyW44CoKXDkUjPzh'])
kupla_id = str(ids_by_spotify_id['7daSp9zXk1dmqNxwKFkL35'])
pinnocchiop_id = str(ids_by_spotify_id['3b7jPCedJ2VH4l4rcOTvNC'])
sugar_and_co_id = str(ids_by_spotify_id['4RCVsWUvlaucR1HTy8Tk7a'])

(tv_room_id, sleigh_bells_id, red_velvet_id, siinamota_id, current_value_id, shiiit_id, skrillex_id, kobaryo_id, loona_id)

('912', '65', '643', '7801598', '57179651', '9318669', '248', '1339641', '515')

In [29]:
a = quok_id
b = dareharu_id
diff = model.wv.get_vector(a) + model.wv.get_vector(b)
diff 

closest = model.wv.similar_by_vector(diff)

def compute_res(close):
    a_dist = model.wv.distance(close, a)
    b_dist = model.wv.distance(close, b)

    return {'spotify_id': spotify_ids_by_id[int(close)],
            'a_dist': a_dist,
            'b_dist': b_dist,
            'abs_dst_diff': abs(a_dist - b_dist)
           }

print("distance: ", model.wv.distance(a, b))

[compute_res(close[0]) for close in closest[:3]]

distance:  0.5944533050060272


[{'spotify_id': '7BjLYT9osbeMveBb7dhlta',
  'a_dist': 0.04651474952697754,
  'b_dist': 0.48716074228286743,
  'abs_dst_diff': 0.4406459927558899},
 {'spotify_id': '0j5kVHxvTgUN4nBIPKCLRJ',
  'a_dist': 0.2172592282295227,
  'b_dist': 0.29585587978363037,
  'abs_dst_diff': 0.07859665155410767},
 {'spotify_id': '1eyg9AJYXvQaXtHyi77WCK',
  'a_dist': 0.09310829639434814,
  'b_dist': 0.452303409576416,
  'abs_dst_diff': 0.35919511318206787}]

In [32]:
def get_most_similar_to_both(id_a, id_b):
    best_distance = 1000000
    best_id = None
    middle = model.wv.get_vector(id_a) + model.wv.get_vector(id_b)
    for id in all_related_artists['id']:
        id = str(id)
        if id == id_a or id == id_b:
            continue
        a_dst = model.wv.distance(id_a, id)
        b_dst = model.wv.distance(id_b, id)
        cost = (a_dst**2 + b_dst**2)
        dist_diff = abs(a_dst - b_dst)
        distance = model.wv.distances(middle, [id])[0]
        if distance < best_distance:
            best_distance = distance
            best_id = id
        
    return best_id

In [47]:
model.wv.save_word2vec_format("work/100k_corpus_4_dims_new_settings_window_7.w2v", "work/100k_corpus_4_dims_new_settings_window_7.vocab")

In [38]:
# (model.wv.get_vector(dareharu_id) ,model.wv.get_vector(deco_id))

def get_val_for_dim(id, dim_ix):
    id = str(id)
    vec = model.wv.get_vector(id)
    return vec[dim_ix]

def get_extremes_for_dim(dim_ix, buckets, count):
    vals = [(spotify_ids_by_id[id], get_val_for_dim(id, dim_ix)) for id in all_related_artists['id']]
    
    vals.sort(key = lambda x: x[1])
    partials_count = buckets - 2
    partial_stride = int(len(vals) / partials_count)
    
    out = []
    out.append(vals[:count])
    
    for partial_ix in range(1, partials_count):
        middle = partial_ix * partial_stride
        out.append(vals[(middle - int(count / 2)):(middle + int(count / 2))])
    
    out.append(vals[-count:])
            
    return out

In [39]:
get_extremes_for_dim(0, 20, 4)

[[('2ovJLSejirvJybNers2YG6', -12.486315),
  ('50BzmPPHNTQjeLW5JvrZJs', -12.470005),
  ('62ABL5pqowZJin1fkheWo7', -12.30272),
  ('4z6dlDfmoZstfKLVC60nM0', -12.292496)],
 [('2tR9SGGAyNGg8A3GBoOFbP', -9.395903),
  ('1OY65G9NJJHsSt1lGLZU6q', -9.395884),
  ('3zrnHI8M8fvS6VmPF8behc', -9.395685),
  ('3AGrDAZx8XN9LhuVxwVN26', -9.3956375)],
 [('30TUV9gTBJ8ffWRMe3O2J0', -7.9888005),
  ('3lSs59gV3uuobbJ6uOtizq', -7.9886775),
  ('6nu39YK7u99qv6JdygWuxG', -7.9886584),
  ('6uqOd0qw9avpT0vCyCZ4DX', -7.9876323)],
 [('13BVQ7HFExOqU7IIGLj5zz', -6.969541),
  ('566KzI0kXa1dQ3em27iQtr', -6.969527),
  ('5RnsHSCPrlCxP9A1NTT7jS', -6.968781),
  ('37bsUoMW0Hbqj1l0MxDkDw', -6.9682755)],
 [('3DDcmHnwKjM4sQ8qm34JMU', -5.623506),
  ('3PS5OcUQlmA1epxoBau3YI', -5.6231227),
  ('5eRKMbpTG8cIUdfomXcJ80', -5.622978),
  ('3VNITwohbvU5Wuy5PC6dsI', -5.6226377)],
 [('0EziqO4Mehje1x6hA1Fg2m', -4.0594497),
  ('6nnspeopmJAG07xOxHmqTu', -4.0592394),
  ('68Uwx1d3EbjDJhYsLjqvIq', -4.059202),
  ('08GQAI4eElDnROBrJRGE0X', -4.0591693

In [58]:
!mv ./*.vocab ./work